# K-Nearest Neighbor Model Implementation


<img src="../images/KNN - Model for Group Assigment.jpg" alt="Alternative text" />

#### IMPORTS


In [1]:
import numpy as np
import pandas as pd
import sklearn.neighbors as skl_nb
import sklearn.model_selection as skl_ms
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

import sys
sys.path.append("..")

from utils.loading_data  import load_to_df_from_csv, get_all_feature_combinations
from utils.knn_functions import find_best_k_with_misclassification_cv, model_iterator_cv

#### STEP 1: LOADING DATA


In [2]:
# Loading the train.csv as the main dataset
data = load_to_df_from_csv("../data/train.csv")

# Column Transformation to lowercase and underscored spaces
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('-', '_')
data.columns = data.columns.str.lower()

X = data.loc[:, data.columns != 'lead']
y = data.loc[:, data.columns == 'lead']

### Exploring data


In [3]:
data.shape

(1039, 14)

In [4]:
data.describe()


,number_words_female,total_words,number_of_words_lead,difference_in_words_lead_and_co_lead,number_of_male_actors,year,number_of_female_actors,number_words_male,gross,mean_age_male,mean_age_female,age_lead,age_co_lead
count,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000
mean,2334.256015,11004.368624,4108.256978,2525.024062,7.767084,1999.862368,3.507218,4561.855630,111.149182,42.353766,35.929588,38.716073,35.486044
std,2157.216744,6817.397413,2981.251156,2498.747279,3.901439,10.406632,2.088526,3417.855987,151.761551,7.817110,8.957193,12.285902,12.046696
min,0.000000,1351.000000,318.000000,1.000000,1.000000,1939.000000,1.000000,0.000000,0.000000,19.000000,11.000000,11.000000,7.000000
25%,904.000000,6353.500000,2077.000000,814.500000,5.000000,1994.000000,2.000000,2139.500000,22.000000,37.480769,29.500000,30.000000,28.000000
50%,1711.000000,9147.000000,3297.000000,1834.000000,7.000000,2000.000000,3.000000,3824.000000,60.000000,42.600000,35.000000,38.000000,34.000000
75%,3030.500000,13966.500000,5227.000000,3364.000000,10.000000,2009.000000,5.000000,5887.500000,143.500000,47.333333,41.500000,46.000000,41.000000
max,17658.000000,67548.000000,28102.000000,25822.000000,29.000000,2015.000000,16.000000,31146.000000,1798.000000,71.000000,81.333333,81.000000,85.000000


In [6]:
feature_combinations = get_all_feature_combinations(X.columns)

results = model_iterator_cv(X, y, feature_combinations, 100)

1 OUT OF 100 ITERATIONS COMPLETED - 1.0%
2 OUT OF 100 ITERATIONS COMPLETED - 2.0%
3 OUT OF 100 ITERATIONS COMPLETED - 3.0%
4 OUT OF 100 ITERATIONS COMPLETED - 4.0%
5 OUT OF 100 ITERATIONS COMPLETED - 5.0%
6 OUT OF 100 ITERATIONS COMPLETED - 6.0%
7 OUT OF 100 ITERATIONS COMPLETED - 7.0%
8 OUT OF 100 ITERATIONS COMPLETED - 8.0%
9 OUT OF 100 ITERATIONS COMPLETED - 9.0%
10 OUT OF 100 ITERATIONS COMPLETED - 10.0%
11 OUT OF 100 ITERATIONS COMPLETED - 11.0%
12 OUT OF 100 ITERATIONS COMPLETED - 12.0%
13 OUT OF 100 ITERATIONS COMPLETED - 13.0%
14 OUT OF 100 ITERATIONS COMPLETED - 14.0%
15 OUT OF 100 ITERATIONS COMPLETED - 15.0%
16 OUT OF 100 ITERATIONS COMPLETED - 16.0%
17 OUT OF 100 ITERATIONS COMPLETED - 17.0%
18 OUT OF 100 ITERATIONS COMPLETED - 18.0%
19 OUT OF 100 ITERATIONS COMPLETED - 19.0%
20 OUT OF 100 ITERATIONS COMPLETED - 20.0%
21 OUT OF 100 ITERATIONS COMPLETED - 21.0%
22 OUT OF 100 ITERATIONS COMPLETED - 22.0%
23 OUT OF 100 ITERATIONS COMPLETED - 23.0%
24 OUT OF 100 ITERATIONS COMP